In [12]:
import os
import pathlib
import pandas as pd

from helpers.age_bindings import *
from helpers.headers import Headers

In [13]:
calculated_avg_mortality_last_3_years = pathlib.Path(f'{os.getcwd()}/../../../output_data/life_table_calculations/Eurostat_avg_mortality_2017_2019(Bulgaria).csv')
mort_df = pd.read_csv(calculated_avg_mortality_last_3_years)

In [14]:
cleaned_avg_population_file = pathlib.Path(f'{os.getcwd()}/../../../cleaned_data/Population_all_ages_2017-2019(Bulgaria).csv')
pop_df = pd.read_csv(cleaned_avg_population_file)

In [15]:
pop_mort = pd.merge(left=mort_df, right=pop_df, on=['Sex', 'Age', 'Location'], suffixes=['_mort', '_pop'])


In [16]:
pop_mort = pop_mort.loc[:, [Headers.location, Headers.sex, Headers.age, Headers.avg_mort, Headers.avg_pop]]
pop_mort[Headers.age] = pop_mort[Headers.age].astype(cat_age_order)
pop_mort.sort_values([Headers.location, Headers.sex, Headers.age], inplace=True)
pop_mort.reset_index(inplace=True)
pop_mort.drop('index', axis=1, inplace=True)

In [19]:
pop_mort[Headers.Interval] = pop_mort[Headers.age].apply(lambda x: INFOSTAT_DECODE_AGE_GROUPS.get(x))
pop_mort[Headers.Mx] = pop_mort.apply(lambda x: x[Headers.avg_mort]/ x[Headers.avg_pop], axis=1)
pop_mort[Headers.ax] = 0.5


In [21]:
pop_mort[Headers.qx] = pop_mort.apply(lambda x: 

(x[Headers.Interval] * x[Headers.Mx])
/
(1 + x[Headers.Interval] * (1 - x[Headers.ax]) * x[Headers.Mx])

if not x[Headers.age] == '(90+)'
else 1,
axis=1)

In [22]:
pop_mort[Headers.px] = pop_mort[Headers.qx].apply(lambda x: 1 - x)

In [23]:
for ind, _ in pop_mort.iterrows():
    if pop_mort.loc[ind, Headers.age] == AGE_BINDINGS.AGE_00_04:
        pop_mort.loc[ind, Headers.l_x] = 100_000
    else:
        index = ind-1
        prev_lx = pop_mort.loc[index, Headers.l_x]
        prev_px = pop_mort.loc[index, Headers.px]
        pop_mort.loc[ind, Headers.l_x] = prev_lx * prev_px

In [25]:
for ind, _ in pop_mort.iterrows():
    if pop_mort.loc[ind, Headers.age] == AGE_BINDINGS.AGE_GE90:
        pop_mort.loc[ind, Headers.d_x] = pop_mort.loc[ind, Headers.l_x]
    else:
        current_lx = pop_mort.loc[ind, Headers.l_x]
        next_lx = pop_mort.loc[ind+1, Headers.l_x]
        pop_mort.loc[ind, Headers.d_x] = current_lx - next_lx

In [27]:
for ind, _ in pop_mort.iterrows():
    if pop_mort.loc[ind, Headers.age] == AGE_BINDINGS.AGE_GE90:
        pop_mort.loc[ind, Headers.L_x] = pop_mort.loc[ind, Headers.l_x] / pop_mort.loc[ind, Headers.Mx]
    else:
        next_lx = pop_mort.loc[ind+1, Headers.l_x]
        current_dx = pop_mort.loc[ind, Headers.d_x]   
        current_ax = pop_mort.loc[ind, Headers.ax]
        current_interval = pop_mort.loc[ind, Headers.Interval]

        pop_mort.loc[ind, Headers.L_x] = current_interval * (next_lx + (current_dx * current_ax))

In [28]:
pop_mort.sort_values([Headers.location, Headers.sex, Headers.age], ascending=False, inplace=True)
pop_mort.reset_index(inplace=True)
pop_mort.drop('index', axis=1, inplace=True)

In [29]:
for ind, _ in pop_mort.iterrows():
    if pop_mort.loc[ind, Headers.age] == AGE_BINDINGS.AGE_GE90:
        pop_mort.loc[ind, Headers.T_x] = pop_mort.loc[ind, Headers.L_x]
    else:
        current_L_x = pop_mort.loc[ind, Headers.L_x]
        previous_T_x = pop_mort.loc[ind - 1, Headers.T_x]
        pop_mort.loc[ind, Headers.T_x] = current_L_x + previous_T_x

In [30]:
pop_mort[Headers.e_x] = pop_mort.apply(lambda x:
                                       x[Headers.T_x] / x[Headers.l_x], 
                                       axis=1).round(2)

In [33]:
pop_mort.sort_values([Headers.location, Headers.sex, Headers.age], inplace=True)
pop_mort.reset_index(inplace=True)
pop_mort.drop('index', axis=1, inplace=True)


In [35]:
calculated_abridged_life_tbl = pathlib.Path(f'{os.getcwd()}/../../../output_data/life_table_calculations/Bulgaria_Abidged_life_table.csv')

pop_mort.to_csv(calculated_abridged_life_tbl, index=False)